In [3]:
import sys, os, pickle
sys.path.append('/scratch/oem214/vanilla-rtrl/')
from cluster import *
from continual_learning import *
from core import *
from dynamics import *
from functions import *
from gen_data import *
from learning_algorithms import *
from optimizers import *
from plotting import *
from wrappers import *
from scipy.ndimage.filters import uniform_filter1d
from sklearn import linear_model

In [4]:
n_seeds = 12
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
difficulties = [1, 3, 5, 7, 10, 13, 16, 19]
macro_configs = config_generator(algorithm=['Only_Output_Weights', 'RTRL',
                                           'UORO', 'KF-RTRL', 'R-KF-RTRL',
                                           'BPTT', 'DNI', 'DNIb',
                                           'RFLO', 'KeRNL'],
                                  difficulty=difficulties)
micro_configs = tuple(product(macro_configs, list(range(n_seeds))))

params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds
np.random.seed(i_job + 800)

In [4]:
params['task'] = 'Add'
params['alpha'] = 1
params['latent_dim'] = None
params['base_learning_rate'] = 1

alpha = params['alpha']

if params['alpha'] == 1:
    n_1, n_2 = 5, 9
    tau_task = 1
if params['alpha'] == 0.5:
    n_1, n_2 = 2, 4
    tau_task = 2

if params['task'] == 'Mimic':

    n_in = 32
    n_hidden = params['difficulty']
    n_out = 32

    W_in_target  = np.random.normal(0, np.sqrt(1/(n_in)), (n_hidden, n_in))
    W_rec_target = np.linalg.qr(np.random.normal(0, 1, (n_hidden, n_hidden)))[0]
    #W_rec_target *= (0.5 + 0.1 * params['difficulty'])
    W_out_target = np.random.normal(0, np.sqrt(1/(n_hidden)), (n_out, n_hidden))
    b_rec_target = np.random.normal(0, 0.1, n_hidden)
    b_out_target = np.random.normal(0, 0.1, n_out)

    alpha = params['alpha']

    rnn_target = RNN(W_in_target, W_rec_target, W_out_target,
                     b_rec_target, b_out_target,
                     activation=tanh,
                     alpha=alpha,
                     output=identity,
                     loss=mean_squared_error)

    task = Mimic_RNN(rnn_target, p_input=0.5, tau_task=tau_task,
                     latent_dim=params['latent_dim'])

elif params['task'] == 'Add':

    n_1 = params['difficulty']
    n_2 = params['difficulty'] + 2
    task = Add_Task(n_1, n_2, deterministic=True, tau_task=tau_task)

data = task.gen_data(1000000, 5000)

n_in = task.n_in
n_hidden = 32
n_out = task.n_out

W_in  = np.random.normal(0, np.sqrt(1/(n_in)), (n_hidden, n_in))
W_rec = np.linalg.qr(np.random.normal(0, 1, (n_hidden, n_hidden)))[0]
W_out = np.random.normal(0, np.sqrt(1/(n_hidden)), (n_out, n_hidden))
W_FB = np.random.normal(0, np.sqrt(1/n_out), (n_out, n_hidden))
b_rec = np.zeros(n_hidden)
b_out = np.zeros(n_out)

alpha = params['alpha']

if params['task'] == 'Add':
    rnn = RNN(W_in, W_rec, W_out, b_rec, b_out,
              activation=tanh,
              alpha=alpha,
              output=softmax,
              loss=softmax_cross_entropy)

if params['task'] == 'Mimic':
    rnn = RNN(W_in, W_rec, W_out, b_rec, b_out,
              activation=tanh,
              alpha=alpha,
              output=identity,
              loss=mean_squared_error)

optimizer = Stochastic_Gradient_Descent(lr=0.0001)
SG_optimizer = Stochastic_Gradient_Descent(lr=0.001)
if params['alpha'] == 1 and params['task'] == 'Add':
    SG_optimizer = Stochastic_Gradient_Descent(lr=0.05)

if params['algorithm'] == 'Only_Output_Weights':
    learn_alg = Only_Output_Weights(rnn)
if params['algorithm'] == 'RTRL':
    learn_alg = RTRL(rnn)
if params['algorithm'] == 'UORO':
    learn_alg = UORO(rnn)
if params['algorithm'] == 'KF-RTRL':
    learn_alg = KF_RTRL(rnn)
if params['algorithm'] == 'R-KF-RTRL':
    learn_alg = Reverse_KF_RTRL(rnn)
if params['algorithm'] == 'BPTT':
    learn_alg = Future_BPTT(rnn, params['difficulty'] + 2)
if params['algorithm'] == 'DNI':
    learn_alg = DNI(rnn, SG_optimizer)
if params['algorithm'] == 'DNIb':
    J_lr = 0.001
    if params['alpha'] == 1 and params['task'] == 'Add':
        J_lr = 0.01
    learn_alg = DNI(rnn, SG_optimizer, use_approx_J=True, J_lr=J_lr,
                    SG_label_activation=tanh, W_FB=W_FB)
    learn_alg.name = 'DNIb'
if params['algorithm'] == 'RFLO':
    learn_alg = RFLO(rnn, alpha=alpha)
if params['algorithm'] == 'KeRNL':
    sigma_noise = 0.0000001
    #kernl_lr = params['base_learning_rate']/sigma_noise
    base_learning_rate = params['base_learning_rate']
    kernl_lr = base_learning_rate/sigma_noise
    KeRNL_optimizer = Stochastic_Gradient_Descent(kernl_lr)
    learn_alg = KeRNL(rnn, KeRNL_optimizer, sigma_noise=sigma_noise,
                      alpha=np.ones(n_hidden)*alpha)

comp_algs = []
monitors = []

sim = Simulation(rnn)
sim.run(data, learn_alg=learn_alg, optimizer=optimizer,
        comp_algs=comp_algs,
        monitors=monitors,
        verbose=True,
        report_accuracy=False,
        report_loss=True)

#Get validation losses
np.random.seed(1)
n_test = 10000
data = task.gen_data(0, n_test)
test_sim = deepcopy(sim)
test_sim.run(data,
             mode='test',
             monitors=['rnn.loss_'],
             verbose=False)
processed_data = np.mean(test_sim.mons['rnn.loss_'])

Progress: 10.0% complete 
Time Elapsed: 14.2s 
Test loss: 0.47568035376981405 

Progress: 20.0% complete 
Time Elapsed: 28.7s 
Test loss: 0.4716595594242823 

Progress: 30.0% complete 
Time Elapsed: 43.3s 
Test loss: 0.4697222507538062 

Progress: 40.0% complete 
Time Elapsed: 57.9s 
Test loss: 0.46817634802057656 

Progress: 50.0% complete 
Time Elapsed: 72.4s 
Test loss: 0.46692422771271447 

Progress: 60.0% complete 
Time Elapsed: 86.9s 
Test loss: 0.46588342723419135 



KeyboardInterrupt: 

In [6]:
### --- SAVE RESULTS -- ###
result = {'sim': sim, 'i_seed': i_seed, 'task': task,
          'config': params, 'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_{}'.format(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [5]:
###Truncate file above
file_name = 'final_fig4c_2'
job_name = 'corr_final_fig4c_2'
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py ../../cluster_main_scripts/{}.py'.format(file_name, job_name))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook final_fig4c_2.ipynb to script
[NbConvertApp] Writing 12333 bytes to final_fig4c_2.py
awk: 1: unexpected character '.'


In [6]:
###Submit job to cluster
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(job_name))
job_id_1 = submit_job('../job_scripts/{}.s'.format(job_name), n_jobs)

rm: cannot remove '/scratch/oem214/learning-dynamics/results/corr_final_fig4c_2/result_*': No such file or directory
sending incremental file list
vanilla-rtrl/
vanilla-rtrl/.DS_Store
vanilla-rtrl/.gitignore
vanilla-rtrl/README.md
vanilla-rtrl/__init__.py
vanilla-rtrl/analyze_main.py
vanilla-rtrl/compare_main.py
vanilla-rtrl/cross_compare_main.py
vanilla-rtrl/jupyter_notebook.py
vanilla-rtrl/test.py
vanilla-rtrl/test_2.py
vanilla-rtrl/.idea/
vanilla-rtrl/.idea/misc.xml
vanilla-rtrl/.idea/modules.xml
vanilla-rtrl/.idea/vanilla-rtrl.iml
vanilla-rtrl/.idea/vcs.xml
vanilla-rtrl/.idea/workspace.xml
vanilla-rtrl/.idea/codeStyles/
vanilla-rtrl/.idea/codeStyles/codeStyleConfig.xml
vanilla-rtrl/.ipynb_checkpoints/
vanilla-rtrl/.ipynb_checkpoints/main-checkpoint.ipynb
vanilla-rtrl/.ipynb_checkpoints/test_interface-checkpoint.ipynb
vanilla-rtrl/__pycache__/
vanilla-rtrl/__pycache__/analysis.cpython-36.pyc
vanilla-rtrl/__pycache__/analysis_funcs.cpython-36.pyc
vanilla-rtrl/__pycache__/analysis_fun

vanilla-rtrl/gen_data/Flip_Flop_Task.py
vanilla-rtrl/gen_data/Mimic_RNN.py
vanilla-rtrl/gen_data/Multi_Task.py
vanilla-rtrl/gen_data/Sensorimotor_Mapping.py
vanilla-rtrl/gen_data/Sequential_MNIST.py
vanilla-rtrl/gen_data/Sine_Wave.py
vanilla-rtrl/gen_data/Task.py
vanilla-rtrl/gen_data/__init__.py
vanilla-rtrl/gen_data/gen_data_utils.py
vanilla-rtrl/gen_data/__pycache__/
vanilla-rtrl/gen_data/__pycache__/Add_Task.cpython-37.pyc
vanilla-rtrl/gen_data/__pycache__/Add_Task.cpython-38.pyc
vanilla-rtrl/gen_data/__pycache__/Delay_Nonmatch_Task.cpython-37.pyc
vanilla-rtrl/gen_data/__pycache__/Delay_Nonmatch_Task.cpython-38.pyc
vanilla-rtrl/gen_data/__pycache__/Fixed_Point_Transition_Task.cpython-37.pyc
vanilla-rtrl/gen_data/__pycache__/Fixed_Point_Transition_Task.cpython-38.pyc
vanilla-rtrl/gen_data/__pycache__/Flip_Flop_Task.cpython-37.pyc
vanilla-rtrl/gen_data/__pycache__/Flip_Flop_Task.cpython-38.pyc
vanilla-rtrl/gen_data/__pycache__/Mimic_RNN.cpython-37.pyc
vanilla-rtrl/gen_data/__pycache_


sent 1,381,820 bytes  received 6,391 bytes  555,284.40 bytes/sec
total size is 1,355,825  speedup is 0.98


In [10]:
###Get job status
get_ipython().system('squeue -u oem214')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          11039313        cm  jupyter   oem214  R    2:32:39      1 cm20


In [16]:
def get_mean_and_ste_curves(results, scalar=True):

    if scalar:
        N_missing = (results==0).sum()
    else:
        N_missing = (results==0).sum()/results.shape[-1]
        
    print('Number of missing cases = {}\n'.format(N_missing))
    
    print('Replacing 0s with nan\n')
    
    results[np.where(results==0)] = np.nan
    
    results = np.squeeze(results)
    if scalar:
        mean_results = np.nanmean(results, -1)
        std_results = np.nanstd(results, -1)
        N_results = (~np.isnan(results)).sum(-1)
        ste_results = std_results/np.sqrt(N_results)
    else:
        mean_results = np.nanmean(results, -2)
        std_results = np.nanstd(results, -2)
        N_results = (~np.isnan(results)).sum(-2)
        ste_results = std_results/np.sqrt(N_results)

    return mean_results, ste_results

In [17]:
learn_algs = ['RTRL', 'DNI', 'UORO','KF-RTRL', 'R-KF-RTRL', 'KeRNL', 'RFLO', 'Only_Output_Weights', 'BPTT', 'DNIb']
colors_dict = {learn_alg: 'C{}'.format(i) for i, learn_alg in enumerate(learn_algs)}

In [ ]:
### --- ADD ALPHA = 1 --- ###

fig, axarr = plt.subplots(1, 2, figsize=(6, 2))

job_name = 'final_fig4c_1'

file_path = os.path.join('/Users/omarschall/cluster_results/vanilla-rtrl/', job_name)

configs_1, results_1, key_order_1, rnn_dict_1 = process_results(file_path)

#results[results == 0] = np.nan
#results = np.squeeze(results)
#median_results = np.nanmedian(results, -1)
#mean_results = np.nanmean(results, -1)
#ste_results = np.nanstd(results, -1)/np.sqrt(len(configs['i_seed']))

mean_results_1, ste_results_1 = get_mean_and_ste_curves(results_1, scalar=True)

job_name = 'final_fig4c_2'

file_path = os.path.join('/Users/omarschall/cluster_results/vanilla-rtrl/', job_name)

configs_2, results_2, key_order_2, rnn_dict_2 = process_results(file_path)

#results_more[results_more == 0] = np.nan
#results_more = np.squeeze(results_more)
#median_results_more = np.nanmedian(results_more, -1)
#mean_results_more = np.nanmean(results_more, -1)
#ste_results_more = np.nanstd(results_more, -1)/np.sqrt(len(configs['i_seed']))

mean_results_2, ste_results_2 = get_mean_and_ste_curves(results_2, scalar=True)

all_mean_results = np.concatenate([mean_results_1, mean_results_2], axis=1)
all_ste_results = np.concatenate([ste_results_1, ste_results_2], axis=1)

all_difficulties = configs_1['difficulty'] + configs_2['difficulty']
all_difficulties_sorted = np.sort(all_difficulties)
sort_idx = np.argsort(all_difficulties)

for i_alg, alg in enumerate(configs_1['algorithm']):
    
    x = all_difficulties_sorted
    mu = all_mean_results[i_alg][sort_idx]
    sigma = all_ste_results[i_alg][sort_idx]
    col = colors_dict[alg]
    axarr[0].plot(x, mu, color=col, linewidth=0.8)
    axarr[0].fill_between(x, mu - sigma, mu + sigma, color=col, alpha=0.3)
    
axarr[0].axhline(y=0.45445, linestyle='--', color='k')
axarr[0].axhline(y=0.66156, linestyle='--', color='k')
axarr[0].set_xticks(all_difficulties_sorted)
axarr[0].set_ylim([0.44, 0.67])

### --- ADD ALPHA = 0.5 --- ###

job_name = 'final_fig4d'

file_path = os.path.join('/Users/omarschall/cluster_results/vanilla-rtrl/', job_name)

configs, results, key_order, rnn_dict = process_results(file_path)

mean_results, ste_results = get_mean_and_ste_curves(results, scalar=True)

#results = np.squeeze(results)
#median_results = np.nanmedian(results, -1)
#mean_results = np.nanmean(results, -1)
#ste_results = np.nanstd(results, -1)/np.sqrt(len(configs['i_seed']))

for i_alg, alg in enumerate(configs['algorithm']):
    
    x = configs['difficulty']
    mu = mean_results[i_alg]
    sigma = ste_results[i_alg]
    col = colors_dict[alg]
    axarr[1].plot(x, mu, color=col, linewidth=0.8)
    axarr[1].fill_between(x, mu - sigma, mu + sigma, color=col, alpha=0.3)
    
axarr[1].axhline(y=0.45445, linestyle='--', color='k')
axarr[1].axhline(y=0.66156, linestyle='--', color='k')
axarr[1].set_xticks(configs['difficulty'])
axarr[1].set_ylim([0.44, 0.67])
fig_name = 'Fig4cd_raw.pdf'
#fig.savefig(os.path.join(figs_path, fig_name), dpi=300, format='pdf')

In [ ]:
### --- MIMIC ALPHA = 1 --- ###

fig, axarr = plt.subplots(1, 2, figsize=(6, 2))

job_name = 'final_fig5c'

file_path = os.path.join('/Users/omarschall/cluster_results/vanilla-rtrl/', job_name)

configs, results, key_order, rnn_dict = process_results(file_path)

mean_results, ste_results = get_mean_and_ste_curves(results)

#results = np.squeeze(results)
#median_results = np.nanmedian(results, -1)
#mean_results = np.nanmean(results, -1)
#ste_results = np.nanstd(results, -1)/np.sqrt(len(configs['i_seed']))

for i_alg, alg in enumerate(configs['algorithm']):
    
    x = np.log2(configs['difficulty'])
    #x = configs['difficulty']
    mu = mean_results[i_alg]
    sigma = ste_results[i_alg]
    col = colors_dict[alg]
    axarr[0].plot(x, mu, color=col, linewidth=0.8)
    axarr[0].fill_between(x, mu - sigma, mu + sigma, color=col, alpha=0.3)
    
axarr[0].set_xticks(x)
axarr[0].set_ylim([0, 0.042])

### --- MIMIC ALPHA = 0.5 --- ###

job_name = 'final_fig5d'

file_path = os.path.join('/Users/omarschall/cluster_results/vanilla-rtrl/', job_name)

configs, results, key_order, rnn_dict = process_results(file_path)

mean_results, ste_results = get_mean_and_ste_curves(results)

for i_alg, alg in enumerate(configs['algorithm']):
    
    if alg == 'Only_Output_Weights':
        continue 
    x = np.log2(configs['difficulty'])
    #x = configs['difficulty']
    mu = mean_results[i_alg]
    sigma = ste_results[i_alg]
    col = colors_dict[alg]
    axarr[1].plot(x, mu, color=col, linewidth=0.8)
    axarr[1].fill_between(x, mu - sigma, mu + sigma, color=col, alpha=0.3)
    
axarr[1].set_xticks(x)
axarr[1].set_ylim([0, 0.013])
fig_name = 'Fig5cd_raw.pdf'
#fig.savefig(os.path.join(figs_path, fig_name), dpi=300, format='pdf')